# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6

In [27]:
import pandas as pd
import xml.etree.ElementTree as ET
import re
import nltk
# nltk.download()
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [28]:
df = pd.read_csv("C:\\Users\\sankl\\Downloads\\recipes_sample.csv")
df = df.iloc[:, 1:3]
df.dropna()
df = df.sample(5)
print("|      id      |  minutes  |\n|--------------------------|")
for idx, row in df.iterrows():
    print(f"|{row[0]:^14}|{row[1]:^11}|")

|      id      |  minutes  |
|--------------------------|
|    188900    |    65     |
|    165892    |    75     |
|    57143     |    10     |
|    187535    |    40     |
|    130100    |   2895    |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [29]:
recipes = pd.read_csv("C:\\Users\\sankl\\Downloads\\recipes_sample.csv")
tree = ET.parse('C:\\Users\\sankl\\Downloads\\steps_sample.xml')
root = tree.getroot()
def show_info(id1):
    recipe = recipes[recipes['id'] == id1].iloc[0]
    steps = root.findall(".//recipe[id='" + str(id1) + "']/steps/step")
    name, author = recipe["name"].title(), f"Автор: {str(recipe['contributor_id'])}"
    minutes = f"Среднее время приготовления: {str(recipe['minutes'])} минут"
    all_steps = "\n".join([f"Step {i} {step.text}" for i, step in enumerate(steps, start = 1)])
    return f"{name}\n\n{all_steps}\n----------\n{author}\n{minutes}"
s = show_info(170895)
print(s)
    

Leeks And Parsnips  Sauteed Or Creamed

Step 1 clean the leeks and discard the dark green portions
Step 2 cut the leeks lengthwise then into one-inch pieces
Step 3 melt the butter in a medium skillet , med
Step 4 heat
Step 5 add the garlic and fry 'til fragrant
Step 6 add leeks and fry until the leeks are tender , about 6-minutes
Step 7 meanwhile , peel and chunk the parsnips into one-inch pieces
Step 8 place in a steaming basket and steam 'til they are as tender as you prefer
Step 9 i like them fork-tender
Step 10 drain parsnips and add to the skillet with the leeks
Step 11 add salt and pepper
Step 12 gently sautee together for 5-minutes
Step 13 at this point you can serve it , or continue on and cream it:
Step 14 in a jar with a screw top , add the half-n-half and arrowroot
Step 15 shake 'til blended
Step 16 turn heat to low under the leeks and parsnips
Step 17 pour in the arrowroot mixture , stirring gently as you pour
Step 18 if too thick , gradually add the water
Step 19 let simme

In [30]:
# assert (
#     show_info(
#         name="george s at the cove black bean soup",
#         steps=[
#             "clean the leeks and discard the dark green portions",
#             "cut the leeks lengthwise then into one-inch pieces",
#             "melt the butter in a medium skillet , med",
#         ],
#         minutes=90,
#         author_id=35193,
#     )
#     == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
# )

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [31]:
recipe_info = show_info(25082)
p=re.findall(r'([0-9]+ hour?s?|[0-9]+ minute?s?)', recipe_info)
p

['20 minutes',
 '10 minutes',
 '2 hours',
 '10 minutes',
 '20 minutes',
 '30 minutes']

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [32]:
pattern = re.compile(r'^this[\w\s]+,\s?but')
matches1 = recipes['description'].str.contains(pattern,na=False)
print(f'Количество рецептов, подходящих под шаблон: {matches1.sum()}\n')
print(recipes.loc[matches1, 'description'].head(3).to_string())

Количество рецептов, подходящих под шаблон: 134

76     this is a great meal eaten the same day ,but e...
183    this was adapted from a recipe i found on the ...
337    this is kind of similar to some of the other v...


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [33]:
recipe_steps = root.findall(".//recipe[id='" + str(72367) + "']/steps/step")
text = "\n".join([str(i+1) + ". " + step.text for i, step in enumerate(recipe_steps)])
print(text)
text = re.sub(r'([0-9]+) / ([0-9]+)',r'\1/\2', text)
print('\n', text)

1. mix butter , flour , 1 / 3 c
2. sugar and 1-1 / 4 t
3. vanilla
4. press into greased 9" springform pan
5. mix cream cheese , 1 / 4 c
6. sugar , eggs and 1 / 2 t
7. vanilla beating until fluffy
8. pour over dough
9. combine apples , 1 / 3 c
10. sugar and cinnamon
11. arrange on top of cream cheese mixture and sprinkle with almonds
12. bake at 350 for 45-55 minutes , or until tester comes out clean

 1. mix butter , flour , 1/3 c
2. sugar and 1-1/4 t
3. vanilla
4. press into greased 9" springform pan
5. mix cream cheese , 1/4 c
6. sugar , eggs and 1/2 t
7. vanilla beating until fluffy
8. pour over dough
9. combine apples , 1/3 c
10. sugar and cinnamon
11. arrange on top of cream cheese mixture and sprinkle with almonds
12. bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [34]:
from bs4 import BeautifulSoup
with open('steps_sample.xml') as f:
    file = BeautifulSoup(f, 'xml')
n_id=file.find_all('id')
steps=file.find_all('steps')
s = ''
for i in steps:
    s+=i.text.lower()
s.replace('\n','')

words = nltk.tokenize.word_tokenize(s)
dict1 = nltk.FreqDist(words)

dictionary = {word: freq for word, freq in dict1.items() if word.isalpha and freq ==1}
dictionary
print(len(dictionary))

9232


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [35]:
maxi = 0
maximum = []
for i in range(recipes.shape[0]):
    try:
        x = len(sent_tokenize(recipes.loc[i].description))
        if x>maxi:
            maxi = x
            maximum = []
            maximum.append(i)
        elif maxi==x:
            maximum.append(i)
    except:
        pass
print(maxi)     

for j in maximum:
    print((' ').join(list(recipes.loc[maximum].description.values)).rstrip())

76
this wonderful icing is used for icing cakes and cookies as well as for borders and art work on cakes.  it makes a delicious filling also between the layers of cakes and under fondant icing.  you can make roses but it takes 3 or more days to dry them depending on the humidity. 

there are many versions of “buttercream” icing. some are made with eggs and all butter.  some varieties, you have to cook your sugar to a softball stage.  others are 100% shortening or a combination of shortening and butter.

each decorator has his or her favorite.  i personally think that the best taste and textured recipe is the one that has you cook your sugar, add to whipped eggs and use pounds of butter per batch. but…. i live in a state that can easily be a 100 degrees for days on end during the summer and you know what butter does on hot days.  it melts!  a greasy puddle of melted icing on a cake plate is not something i want to look at or eat.  

your top notch decorators have a few options we don’t.

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [36]:
from nltk import pos_tag, word_tokenize

def f(strr):
    sentence = nltk.tokenize.word_tokenize(strr)
    f = ''
    f1 = ''
    part_of_speech =  [j[1] for j in pos_tag(word_tokenize(strr))]

    for i in range(len(sentence)):
        d =max(len(sentence[i]),len(part_of_speech[i])) 
        f1+=f'{sentence[i]:^{d}}'+' '
        f+=f'{part_of_speech[i]:^{d}}'+' '
        print(f)
        print(f1)
    
    
f(' I  omitted the raspberries and added strawberries instead')
f((' ').join(list(recipes[recipes['id']==241106].description.values)))

PRP 
 I  
PRP   VBD   
 I  omitted 
PRP   VBD   DT  
 I  omitted the 
PRP   VBD   DT      NNS     
 I  omitted the raspberries 
PRP   VBD   DT      NNS     CC  
 I  omitted the raspberries and 
PRP   VBD   DT      NNS     CC   VBD  
 I  omitted the raspberries and added 
PRP   VBD   DT      NNS     CC   VBD      NNS      
 I  omitted the raspberries and added strawberries 
PRP   VBD   DT      NNS     CC   VBD      NNS        RB    
 I  omitted the raspberries and added strawberries instead 
 DT   
these 
 DT   VBP 
these are 
 DT   VBP DT 
these are a  
 DT   VBP DT   RB   
these are a  really 
 DT   VBP DT   RB    JJ  
these are a  really good 
 DT   VBP DT   RB    JJ   NN   
these are a  really good quick 
 DT   VBP DT   RB    JJ   NN    NN  
these are a  really good quick meal 
 DT   VBP DT   RB    JJ   NN    NN  TO 
these are a  really good quick meal to 
 DT   VBP DT   RB    JJ   NN    NN  TO  VB  
these are a  really good quick meal to make 
 DT   VBP DT   RB    JJ   NN    NN  TO